In [3]:
%%capture
# update or install the necessary libraries
!pip install --upgrade google-search-results


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain.agents import Tool, load_tools
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentExecutor

from virtual_assistant.tools import BrowsingTool, RetrivalTool
from virtual_assistant.agents.chat_agent import ConversationalChatAgent
from virtual_assistant.utilities import GoogleSerperAPIWrapper, initialize_pinecone

/Users/al002/github/virtual_assistant/virtual-assistant-chain/.venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
initialize_pinecone()

True

In [4]:
embeddings = OpenAIEmbeddings()

In [5]:
db = Pinecone.from_existing_index(index_name='personal-knowledgebase', embedding=embeddings)
retriever = db.as_retriever()

In [6]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [7]:
# tool_names = ['google-serper']
# tools = load_tools(tool_names, llm=llm)

tools = [RetrivalTool(retriver=retriever), BrowsingTool(serper=GoogleSerperAPIWrapper())]

In [8]:
llm=ChatOpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="chat-conversational-react-description", verbose=True, memory=memory)

In [9]:
agent = ConversationalChatAgent.from_llm_and_tools(llm=llm, tools=tools)

In [10]:
executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)

In [11]:
executor.run("what are some good dinners to make this week, if i like japanese food?")



> Entering new AgentExecutor chain...
Action: Browsing internet tool
Plan:
- [ ] Search for "Japanese dinner recipes"
- [ ] Look for recipes that fit the user's dietary restrictions
- [ ] Choose 3-5 recipes that look appealing
What I Did: None
Action Input: Japanese dinner recipes
Observation: Best Sushi Recipes to Make at Home 21 Most Popular Japanese Desserts15 Best Matcha RecipesNabemono: A Guide to Japanese Hot Pot
More Resources on Japanese Cooking
Japanese Pantry Ingredients to Add to Your Shopping ListIngredient Substitutions for Japanese Cooking Japanese Grocery Stores around the WorldEssential Japanese Cooking Tips for Beginners

Editor’s Note: This post was originally published on March 22, 2020. It’s been republished with new content on January 25, 2023.

MainRoundup 
Published:  Jan 25, 2023Updated:  Apr 10, 2023Written by Nami 













Meet the Author
 Nami 
I'm Nami, a Japanese home cook based in San Francisco. Have fun exploring the 1000+ classic & modern Japanese

'Here are some Japanese dinner recipes you might like to try: Japanese Tomato Curry, Oyakodon (Chicken and Egg Rice Bowl), and Teriyaki Salmon. You can find the recipes on the Just One Cookbook website.'

In [20]:
executor.run("list 5 langchain agent tools from langchain documents")



> Entering new AgentExecutor chain...
Action: knowledgebase retrival tool
Plan:
- [ ] Open the Langchain AGI knowledgebase
- [ ] Search for the Langchain AGI tools section
- [ ] Retrieve the list of tools
What I Did: None
Action Input: "List of Langchain AGI tools from Langchain documents"
Observation: Conversational Agent with Tools (Langchain AGI)
Langchain AGI (23rd Dec 2022)
Proprietary#
Daimon
A chat-based AI personal assistant with long-term memory about you.
AI Assisted SQL Query Generator
An app to write SQL using natural language, and execute against real DB.
Clerkie
Stack Tracing QA Bot to help debug complex stack tracing (especially the ones that go multi-function/file deep).
Sales Email Writer
By Raza Habib, this demo utilizes LangChain + SerpAPI + HumanLoop to write sales emails. Give it a company name and a person, this application will use Google Search (via SerpAPI) to get more information on the company and the person, and then write them a sales message.
Question-An

'Conversational Agent with Tools (Langchain AGI), AI Assisted SQL Query Generator, Clerkie, Sales Email Writer, and Question-Answering on a Web Browser.'

In [13]:
agent_chain.run(input="tell me who won the world cup in 2022?")



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "who won the world cup 2022"
}
Observation: 2022 World Cup / Champion:  Argentina national football team
Thought:{
    "action": "Final Answer",
    "action_input": "Argentina national football team won the 2022 World Cup."
}

> Finished chain.


'Argentina national football team won the 2022 World Cup.'

In [11]:
agent_chain.run('Oscar best movie in 2023.')



> Entering new AgentExecutor chain...
{
    "action": "Browsing internet tool",
    "action_input": "What was the Oscar best movie in 2023"
}
Observation: Academy Award for Best Picture / Winners (2023) :  Everything Everywhere All at Once
Thought:{
    "action": "Final Answer",
    "action_input": "The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."
}

> Finished chain.


"The Academy Award for Best Picture winner in 2023 was 'Everything Everywhere All at Once'."